In [1]:
def byte2str(n, d = 2):
    s = 'B'
    if n >= 1024:
        n/=1024
        s = 'K'
    if n >= 1024:
        n/=1024
        s = 'M'
    if n >= 1024:
        n/=1024
        s = 'G'
    if n >= 1024:
        n/=1024
        s = 'T'
    return '{:.2f}{}'.format(n,s)

In [2]:
dic = {'name': 'MIB',
'details': {'cpu':'XEON E5 6666','cpu_cores':8, 'memory':"16GB*4&2666MHz",'gpu':'RTX 2080 Ti*2'},
'timeStamp': 1932846198,
'cpu_total': 0.73,
'cpu_per_core': [0.0,0.43,0.45,0.56,0.89,0.01,0.01,0.02],
'memory_status': '562M/38.0G',
'memory_percentage': 0.07,
'top_cpu_user': 'agent_h',
'gpu_status':[
                {
                'gpu_no': 0,
                'gpu_name': 'RTX 2080 Ti',
                'gpu_memory_status': '12M/8119M',
                'gpu_util': 0.78,
                'gpu_memory': 0.23,
                'gpu_temperature': 48,
                'top_gpu_user': 'agent_n',
                },
                {
                'gpu_no': 1,
                'gpu_name': 'RTX 2080 Ti',
                'gpu_memory_status': '3434M/8119M',
                'gpu_util': 0.33,
                'gpu_memory': 0.75,
                'gpu_temperature': 78,
                'top_gpu_user': 'alien_z',
                }
            ]
}

In [3]:
import psutil

### timeStamp

### cpu_total

In [4]:
psutil.cpu_percent(interval=.5, percpu=False)

57.2

### cpu_per_core

In [5]:
psutil.cpu_percent(interval=.5, percpu=True)

[0.0,
 0.0,
 97.3,
 70.6,
 72.0,
 0.0,
 70.0,
 70.0,
 70.6,
 70.6,
 0.0,
 0.0,
 0.0,
 100.0,
 0.0,
 0.0]

### memory_status

In [6]:
'{}/{}'.format(byte2str(psutil.virtual_memory().used),byte2str(psutil.virtual_memory().total))

'3.42G/38.99G'

### memory_percentage

In [7]:
psutil.virtual_memory().percent

10.3

In [8]:
def get_plist():
    p_list = []
    p_list_dict = {}
    for i in psutil.process_iter():
    #     print(i.as_dict())
        p_list.append((i.as_dict()['cpu_percent'],i.as_dict()['username'],i.as_dict()['name']))
        p_list_dict[str(i.as_dict()['pid'])]=(i.as_dict()['cpu_percent'],i.as_dict()['username'],i.as_dict()['name'])
    return p_list,p_list_dict
p_list,p_list_dict=get_plist()
print('{}:{}:{}%'.format(sorted(p_list)[-1][1],sorted(p_list)[-1][2],sorted(p_list)[-1][0]))
# print(sorted(p_list)[::-1])


whoopsie:whoopsie:0.0%


In [9]:
from pynvml import *
nvmlInit()

In [10]:
from pynvml import *
nvmlInit()
print("Driver Version:", nvmlSystemGetDriverVersion())
deviceCount = nvmlDeviceGetCount()
for i in range(deviceCount):
    handle = nvmlDeviceGetHandleByIndex(i)
    print("Device", i, ":", nvmlDeviceGetName(handle))
    print("Temp",i,";",nvmlDeviceGetTemperature(handle,0))
    print("gpu_util", i, ":", nvmlDeviceGetUtilizationRates(handle).gpu)
    print("gpu_mem_util", i, ":", 100*nvmlDeviceGetMemoryInfo(handle).used/nvmlDeviceGetMemoryInfo(handle).total)
    print("gpu_total_mem", i, ":", byte2str(nvmlDeviceGetMemoryInfo(handle).total))
    print("gpu_used_mem", i, ":", byte2str(nvmlDeviceGetMemoryInfo(handle).used))
    procs = nvmlDeviceGetComputeRunningProcesses(handle)
    
    mem = -1
    user = None
    for p in procs:
        if p.usedGpuMemory > mem:
            user = '{}:{}:{}'.format(p_list_dict[str(p.pid)][1],
                                     p_list_dict[str(p.pid)][2],
                                     byte2str(p.usedGpuMemory))
    print(user)


#     print(handle)


Driver Version: b'430.40'
Device 0 : b'GeForce RTX 2080 Ti'
Temp 0 ; 46
gpu_util 0 : 100
gpu_mem_util 0 : 36.1525939958369
gpu_total_mem 0 : 10.76G
gpu_used_mem 0 : 3.89G
agent_h:python:3.87G
Device 1 : b'GeForce RTX 2080 Ti'
Temp 1 ; 32
gpu_util 1 : 0
gpu_mem_util 1 : 0.15257130865345894
gpu_total_mem 1 : 10.76G
gpu_used_mem 1 : 16.81M
None


In [93]:
def get_gpu_status(p_list_dict):
    gpu_status_list = []
    nvmlInit()
    deviceCount = nvmlDeviceGetCount()
    for i in range(deviceCount):
        handle = nvmlDeviceGetHandleByIndex(i)
        procs = nvmlDeviceGetComputeRunningProcesses(handle)
        mem = 0
        user = None
        for p in procs:
            if p.usedGpuMemory > mem:
                user = '{}:{}'.format(p_list_dict[str(p.pid)][1],
                                         byte2str(p.usedGpuMemory))
        status = {
                       'gpu_no': i,
                       'gpu_name': nvmlDeviceGetName(handle).decode('utf-8'),
                       'gpu_memory_status': '{}/{}'.format(byte2str(nvmlDeviceGetMemoryInfo(handle).used),byte2str(nvmlDeviceGetMemoryInfo(handle).total)),
                       'gpu_util': nvmlDeviceGetUtilizationRates(handle).gpu,
                       'gpu_memory': round(100*nvmlDeviceGetMemoryInfo(handle).used/nvmlDeviceGetMemoryInfo(handle).total,2),
                       'gpu_temperature': nvmlDeviceGetTemperature(handle,0),
                       'top_gpu_user': user,
                   }
        gpu_status_list.append(status)
    return gpu_status_list
# print(get_gpu_status(p_list_dict))

In [192]:
import time
import platform
def get_sys_info():
    sys_dic = {'name': 'MIB',
               'details': {'cpu':'XEON E5 6666','cpu_cores':8, 'memory':"16GB*4&2666MHz",'gpu':'RTX 2080 Ti*2'},
               'timestamp': 1932846198,
               'cpu_total': 0.73,
               'cpu_per_core': [0.0,0.43,0.45,0.56,0.89,0.01,0.01,0.02],
               'memory_status': '562M/38.0G',
               'memory_percentage': 0.07,
               'top_cpu_user': 'agent_h',
               'gpu_status':[
                   {
                       'gpu_no': 0,
                       'gpu_name': 'RTX 2080 Ti',
                       'gpu_memory_status': '12M/8119M',
                       'gpu_util': 0.78,
                       'gpu_memory': 0.23,
                       'gpu_temperature': 48,
                       'top_gpu_user': 'agent_n',
                   },
                   {
                       'gpu_no': 1,
                       'gpu_name': 'RTX 2080 Ti',
                       'gpu_memory_status': '3434M/8119M',
                       'gpu_util': 0.33,
                       'gpu_memory': 0.75,
                       'gpu_temperature': 78,
                       'top_gpu_user': 'alien_z',
                   }
               ]
              }
    sys_dic['name']=get_ip()
    sys_dic['timestamp']=time.time()
    sys_dic['cpu_total']=psutil.cpu_percent(interval=.5, percpu=False)
    sys_dic['cpu_per_core']=psutil.cpu_percent(interval=.5, percpu=True)
    sys_dic['memory_status']='{}/{}'.format(byte2str(psutil.virtual_memory().used),byte2str(psutil.virtual_memory().total))
    sys_dic['memory_percentage'] = psutil.virtual_memory().percent
    p_list,p_list_dict=get_plist()
    sys_dic['top_cpu_user']='{}:{}:{}%/{}%'.format(sorted(p_list)[-1][1],sorted(p_list)[-1][2],sorted(p_list)[-1][0],
                                                   str(len(sys_dic['cpu_per_core'])*100))
    sys_dic['gpu_status']=get_gpu_status(p_list_dict)
    sys_dic['details']={'cpu':platform.processor(),
                        'cpu_cores':str(len(sys_dic['cpu_per_core'])),
                        'memory':byte2str(psutil.virtual_memory().total),
                        'gpu':[x['gpu_name'] for x in sys_dic['gpu_status']]
                                          
    }
    return sys_dic
    

In [193]:
dic = get_sys_info()
dic


{'name': '10.229.0.6',
 'details': {'cpu': 'x86_64',
  'cpu_cores': '16',
  'memory': '38.99G',
  'gpu': ['GeForce RTX 2080 Ti', 'GeForce RTX 2080 Ti']},
 'timestamp': 1567863777.972158,
 'cpu_total': 74.0,
 'cpu_per_core': [58.0,
  35.3,
  85.7,
  45.1,
  88.2,
  98.0,
  49.0,
  36.0,
  26.0,
  44.0,
  44.0,
  36.0,
  44.0,
  47.1,
  34.0,
  30.0],
 'memory_status': '6.37G/38.99G',
 'memory_percentage': 17.9,
 'top_cpu_user': 'agent_h:python:394.3%/1600%',
 'gpu_status': [{'gpu_no': 0,
   'gpu_name': 'GeForce RTX 2080 Ti',
   'gpu_memory_status': '4.77G/10.76G',
   'gpu_util': 98,
   'gpu_memory': 44.36,
   'gpu_temperature': 48,
   'top_gpu_user': 'agent_h:4.76G'},
  {'gpu_no': 1,
   'gpu_name': 'GeForce RTX 2080 Ti',
   'gpu_memory_status': '997.81M/10.76G',
   'gpu_util': 56,
   'gpu_memory': 9.06,
   'gpu_temperature': 41,
   'top_gpu_user': 'agent_m:981.00M'}]}

In [87]:
import platform
import json
import requests
import time
from get_sys_info import *

In [17]:
dic = get_sys_info()
data = json.dumps(dic)
url = 'http://3.14.87.92:5000/add_machine'
header = {'Content-Type': "application/json"}
response = requests.post(url, data=data, headers=header)
print(response.content.decode('utf-8'))


ConnectionError: HTTPConnectionPool(host='3.14.87.92', port=5000): Max retries exceeded with url: /add_machine (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb954076690>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [98]:
def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 50, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
#     percent = ""
    filledLength = int(length * iteration // total)
#     print(filledLength)
    bar = fill * filledLength + '-' * (length - filledLength)
    return str('%s |%s| %s%% %s' % (prefix, bar, percent, suffix))
    # Print New Line on Complete
    
printProgressBar (55, 100, prefix = '', suffix = '', decimals = 1, length = 10, fill = '█')

' |█████-----| 55.0% '

In [203]:
import datetime
ip_str = {
    '10.229.0.4':'KAIJU',
    '10.229.0.6':'MIB',
    '10.229.0.8':'JAEGER',
    '10.229.2.6':'LAIDT1',
    '10.229.2.7':'LAIDT2',
    '10.229.2.8':'LAIDT3',
    '10.229.2.9':'LAIDT4',
    '10.229.2.10':'LAIDT5',
    '10.229.2.11':'LAIDT6',
    '10.229.2.12':'LAIDT7',
    '10.229.2.13':'LAIDT8',
}
def get_status_string_tiny(status_dict, mode = "tiny_wide"):
    s = ''
#     s += "{:=^119}\n".format(" "+status_dict["name"]+": "+datetime.datetime.fromtimestamp(status_dict["timestamp"]).strftime('%c')+" ")
    name = status_dict["name"]
    if name in ip_str:
        name = ip_str[name]
    s += "{:=^119}\n".format(" "+name+": "+status_dict["timestamp"]+" ")
    s += 'CPU: {: <60}   {: >51}\n'.format(printProgressBar(status_dict["cpu_total"],100,length = 20, suffix = str(status_dict["top_cpu_user"])),
                                    "MEM: "+printProgressBar(status_dict["memory_percentage"],100,length = 20, suffix = status_dict["memory_status"]))
#     s += 'MEM: ' + printProgressBar(status_dict["memory_percentage"],100,length = 15, suffix = status_dict["memory_status"])
    for i,g in enumerate(status_dict["gpu_status"]):
        s += '{}: {}, Util:{: <26} Mem: {: <42} {: >15}\n'.format(str(i),
                                    g["gpu_name"],
                                    printProgressBar(g["gpu_util"],100,length = 15),
                                    printProgressBar(g["gpu_memory"],100,suffix = g["gpu_memory_status"],length = 15),
                                                     str(g["top_gpu_user"]))
    return s
# dic = get_sys_info()
# print(get_status_string_tiny(dic))
# print(get_status_string_tiny(dic))


In [204]:
url = 'http://3.14.87.92/get_machines'
response = requests.get(url)
resp = json.loads(response.content.decode('utf-8'))

# print(json.loads(resp))
info_dict = {}
for re in resp:
    if re["name"] not in info_dict:
        info_dict[re["name"]] = re
    else:
        if re["timestamp"]>info_dict[re["name"]]["timestamp"]:
            info_dict[re["name"]] = re
# print(info_dict)
keys = sorted(list(info_dict.keys()))
for i in keys:
    print(get_status_string_tiny(info_dict[i]))
# print(resp)

=========================================== MIB: 2019-09-07 13:57:00.369399 ===========================================
CPU:  |██------------------| 14.4% agent_h:python:427.5%/1600%          MEM:  |██------------------| 10.7% 3.56G/38.99G
0: GeForce RTX 2080 Ti, Util: |██████████████-| 99.0%   Mem:  |█████████------| 61.8% 6.65G/10.76G        agent_h:6.63G
1: GeForce RTX 2080 Ti, Util: |---------------| 0.0%    Mem:  |---------------| 0.1% 16.81M/10.76G                 None

========================================= LAIDT5: 2019-09-07 13:56:48.932120 ==========================================
CPU:  |--------------------| 0.2% alien_h:python:31.7%/1200%            MEM:  |██████--------------| 32.4% 4.44G/15.68G
0: GeForce GTX 1070 Ti, Util: |---------------| 0.0%    Mem:  |---------------| 4.3% 346.00M/7.92G                 None



In [159]:
data = json.dumps(dic)
url = 'http://127.0.0.1:5000/add_machine'
header = {'Content-Type': "application/json"}
response = requests.post(url, data=data, headers=header)
print(response.content.decode('utf-8'))
url = 'http://127.0.0.1:5000/get_machines'
response = requests.get(url)
print(response.content.decode('utf-8'))

ConnectionError: HTTPConnectionPool(host='127.0.0.1', port=5000): Max retries exceeded with url: /add_machine (Caused by NewConnectionError('<urllib3.connection.HTTPConnection object at 0x7fb93e7156d0>: Failed to establish a new connection: [Errno 111] Connection refused'))

In [68]:
import datetime
datetime.datetime.fromtimestamp(time.time()).strftime('%c')

'Fri Sep  6 18:58:09 2019'

In [191]:
import socket

def get_ip():
    s = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)
    s.connect(("8.8.8.8", 80))
    ip = s.getsockname()[0]
    s.close()
    return ip